<a href="https://colab.research.google.com/github/meirarc/Market_Price/blob/main/Market_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
from __future__ import print_function
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json


In [94]:
enable_print = False
filename = "/content/drive/MyDrive/Colab/Market_Price.csv"

In [91]:
def print(*args, **kwargs):
  if enable_print:
    return __builtins__.print(*args, **kwargs)

In [85]:
def compare_price(current, new):
  print("(compare_price): current: ",current, " is blank: ", current == "")
  print("(compare_price): new: ", new, " is blank: ", new == "")
  print("(compare_price:) ",current, new)

  current = float(str(current).replace(",",".")) if current != "" and current is not None else 0 
  new = float(str(new).replace(",",".")) if new != "" and new is not None else 0

  status = ""
  status = "Same Price" if new == current else status  
  status = "More Expensive" if new > current else status   
  status = "On Sale" if new < current else status
  return status

In [43]:
def marche(item):
    webpage = requests.get(item["URL"])
    soup = BeautifulSoup(webpage.content, "html.parser")
    info = soup.find(attrs={"class":"name-and-info"}).p.get_text()
    
    product_name = soup.find(attrs={"class":"product-name"}).get_text()
    product_price = float(info.split("R$ ",1)[1].split("/",1)[0].replace(",","."))
    product_unit = info.split("R$ ",1)[1].split("/",1)[1]

    if product_unit.strip() == "kg":
      size = "1"
      price_kg = product_price
    else:
      size = product_name.split(" ")[-1]
      if size[-1] == "g":
        size = float(size.replace("g", ""))/1000
        price_kg = round(product_price/size, 2)
        product_unit = "g"
      else:
        if product_unit.strip() == "un":
          size = "3"
          price_kg = product_price * 3
        else:
          size = ""
          price_kg = ""

    status = compare_price(item["Product Price per Kg"], price_kg)
    
    item["Product Name"] = product_name if product_name is not None else ""
    item["Product Price"] = "{:.2f}".format(product_price) if product_price is not None else ""
    item["Product Unit"] = product_unit if product_unit is not None else ""
    item["Product Size"] = "{:.3f}".format(float(size)) if size is not None else ""
    item["Product Price per Kg"] = "{:.2f}".format(price_kg) if price_kg is not None else ""
    item["Status"] = status if status is not None else ""

    return item

In [45]:
def oba(item):

  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = ""
  product_price = ""
  product_unit = ""
  size = ""
  price_kg = ""

  product_name = soup.find(attrs={"class":re.compile("ProductDetailsStyles")})
  product_name = product_name if product_name is not None else ""
  
  product_price = soup.find(attrs={"class":re.compile("ProductDetailsStyles")})
  product_price = product_price if product_price is not None else ""
  
  product_unit = ""

  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""
  
  return item

In [46]:
def hakuo(item):
  
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = ""
  product_price = ""
  product_unit = ""
  size = ""
  price_kg = ""

  product_name = item["URL"].split("/")[-1].replace("-", " ")
  product_name = product_name if product_name is not None else ""
  
  product_price = soup.find(attrs={"class":re.compile("area-preco-detalhe-produto")})
  print(product_price)
  product_price = product_price if product_price is not None else ""
  
  product_unit = ""

  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""
  
  return item

In [95]:
def sams(item):
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = soup.find(attrs={"class":re.compile("productBrand")})
  product_name = product_name.get_text().strip() if product_name is not None else "Error Product Name"
  print("(sams) product_name: ",product_name)

  product_price  = soup.find(attrs={"class":re.compile("productShowCasePricePdp")})
  product_price_kg  = soup.find(attrs={"class":re.compile("productShowCasePriceKgPdp")})
  
  price_kg = 0
  price = 0
  unit = ""
  
  if product_price_kg is not None:
    clean_price_kg = product_price_kg.get_text().strip()
    if clean_price_kg != "":
      price_kg = clean_price_kg.split(" ")[-1].replace(u'\xa0', ' ').split("$")[-1].replace(",",".").strip()
      unit = clean_price_kg.split(" ")[0]
  print("(sams) price_kg: ",price_kg)
  print("(sams) unit: ",unit)

  if product_price is not None:
    clean_price = product_price.get_text().strip()
    if clean_price != "":
      price = clean_price.split("$")[-1].replace(",",".").strip()
  print("(sams) price: ",price)

  product_price = price_kg if price_kg != 0 else price
  product_price = product_price if product_price is not None else ""
  print("(sams) product_price: ",product_price)

  product_unit = unit if unit != "" else ""
  product_unit = product_unit if product_unit is not None else ""
  print("(sams) product_unit: ", product_unit)

    
  if product_unit.strip() == "Preço/Kg":
    size = "1"
    price_kg = product_price
    product_unit = "kg"
  else:
    unit = product_name.strip().split(" ")[-1]
    product_unit = unit
    
    if product_unit == "Unidade":
      size = "3"
      price_kg = "{:.2f}".format(float(product_price) * 3) if product_price != "" else ""
      product_unit = "un" 
    else:
      if product_unit[-2:].lower() == "kg": 
        size = product_unit[:-2].replace(",", ".")
        price_kg = round(float(product_price) / float(size), 2)
        product_unit = "kg"
      else:
        print(product_unit)
        if product_unit.strip()[-1].lower() == "g": 
          size = float(product_unit[:-1])/1000
          price_kg = float(product_price) / size if product_price != "" else ""
          product_unit = "g"
        else:
          if product_name.strip() == "Batata Pacote":
              size = "2"
              price_kg = float(product_price) / 2
              product_unit = "kg"
          else:
            if product_name.strip() == "Agrião Verdureira":
              size = "3"
              price_kg = float(product_price) * 2
              product_unit = "un"
            else:
              size = ""
              price_kg = ""
              product_unit = ""

  print("(sams) current, item[Product Price per Kg]: ", item["Product Price per Kg"] == "")
  print("(sams) new, price_kg: ", price_kg)
  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""

  return item

In [68]:
def paodeacucar(item):

  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}
  webpage = requests.get(item["URL"], headers=headers)
  soup = BeautifulSoup(webpage.content, "html.parser")
  
  product_name = soup.find(attrs={"class":"textstyles__TextComponent-w4b5ef-0 iISkjF"}).get_text().strip()
  product_unit = ""
  size = product_name.split(" ")[-1]

  product_price = soup.find(attrs={"class":re.compile("CurrentPrice")})
  product_price = float(product_price.p.get_text().split("R$ ",1)[1].replace(",",".")) if product_price is not None else ""


  if size[-2].lower() == "k":
    size = float(size.lower().replace("kg", "").replace(",","."))
    price_kg = round(product_price/size, 2) if product_price != "" else ""
    product_unit = "kg"
  else: 
    if size[-1] == "g":
      size = float(size.replace("g", ""))
      price_kg = round(product_price/size*1000, 2) if product_price != "" else ""
      product_unit = "g"
    else:
      size = "3"
      product_unit = "un"
      price_kg = product_price * 3

  status = compare_price(item["Product Price per Kg"], price_kg)

  item["Product Name"] = product_name if product_name is not None else ""
  item["Product Price"] = product_price if product_price is not None else ""
  item["Product Unit"] = product_unit if product_unit is not None else ""
  item["Product Size"] = size if size is not None else ""
  item["Product Price per Kg"] = price_kg if price_kg is not None else ""
  item["Status"] = status if status is not None else ""

  return item

In [70]:
df = pd.read_csv(filename, sep=";", encoding = "utf-8")
df["Product Price per Kg"] = df["Product Price per Kg"].str.replace(",",".")
df["Product Price"] = df["Product Price"].str.replace(",",".")
df["Product"] = df["Product"].str.replace(u'\xa0', u' ')
df = df.fillna("")
df

,Product,Market,Product Price per Kg,URL,Product Name,Product Price,Product Unit,Product Size,Status,new URL
0,batata doce rosada,Sams Club,3.98,https://www.samsclub.com.br/batata-doce-roxa-p...,,,,,,https://www.samsclub.com.br/batata-doce-roxa-p...
1,batata doce rosada,Pao de Acucar,6,https://www.paodeacucar.com/produto/80528/bata...,,,,,,https://www.paodeacucar.com/produto/80528/bata...
2,batata doce rosada,Giga,3.9,,,,,,,
3,batata doce rosada,Oba,5.99,,,,,,,
4,batata doce rosada,Hakuo,5.59,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
120,quiabo,St. Marche,39.9,https://www.marche.com.br/produtos/quiabo-sele...,Quiabo Selecionado,39.9,kg,1,On Sale,
121,ervilha torta,Pao de Acucar,22.3,https://www.paodeacucar.com/produto/87149/ervi...,,0,,0,,https://www.paodeacucar.com/produto/87149/ervi...
122,ervilha torta,St. Marche,79.5,https://www.marche.com.br/produtos/ervilha-con...,,0,,0,,https://www.marche.com.br/produtos/ervilha-con...
123,palmito pupunha,St. Marche,53.3,,,0,,0,,


In [49]:
string = df.to_json(orient = 'records')
data = json.loads(string)

In [93]:
for item in data:
  if item["URL"] is not None and item["URL"] != "":
    marche(item) if item["Market"] == "St. Marche" else None
    sams(item) if item["Market"] == "Sams Club" else None
    paodeacucar(item) if item["Market"] == "Pao de Acucar" else None
    #oba(item) if item["Market"] == "Oba" else None
    #hakuo(item) if item["Market"] == "Hakuo" else None

(compare_price): current:  5.26  is blank:  False
(compare_price): new:  5.26  is blank:  False
compare_price:  5.26 5.26
(compare_price): current:  6.7  is blank:  False
(compare_price): new:  6.7  is blank:  False
compare_price:  6.7 6.7
(compare_price): current:  7.99  is blank:  False
(compare_price): new:  7.99  is blank:  False
compare_price:  7.99 7.99
(compare_price): current:  5.49  is blank:  False
(compare_price): new:  5.49  is blank:  False
compare_price:  5.49 5.49
(compare_price): current:  6.99  is blank:  False
(compare_price): new:  6.99  is blank:  False
compare_price:  6.99 6.99
(compare_price): current:  8.99  is blank:  False
(compare_price): new:  8.99  is blank:  False
compare_price:  8.99 8.99


KeyboardInterrupt: ignored

In [79]:
data_df = pd.DataFrame(data)
data_df

,Product,Market,Product Price per Kg,URL,Product Name,Product Price,Product Unit,Product Size,Status,new URL
0,batata doce rosada,Sams Club,5.26,https://www.samsclub.com.br/batata-doce-roxa-p...,Batata Doce Roxa Pacote,5.26,kg,1,Same Price,https://www.samsclub.com.br/batata-doce-roxa-p...
1,batata doce rosada,Pao de Acucar,6.7,https://www.paodeacucar.com/produto/80528/bata...,Batata Doce Rosada 400g,2.68,g,400.0,Same Price,https://www.paodeacucar.com/produto/80528/bata...
2,batata doce rosada,Giga,3.9,,,,,,,
3,batata doce rosada,Oba,5.99,,,,,,,
4,batata doce rosada,Hakuo,5.59,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
120,quiabo,St. Marche,39.90,https://www.marche.com.br/produtos/quiabo-sele...,Quiabo Selecionado,39.90,kg,1.000,Same Price,
121,ervilha torta,Pao de Acucar,63.96,https://www.paodeacucar.com/produto/87149/ervi...,Ervilha Torta QUALITÁ Bandeja 250g,15.99,g,250.0,More Expensive,https://www.paodeacucar.com/produto/87149/ervi...
122,ervilha torta,St. Marche,34.63,https://www.marche.com.br/produtos/ervilha-con...,Ervilha Congelada BONDUELLE 300g,10.39,g,0.300,Same Price,https://www.marche.com.br/produtos/ervilha-con...
123,palmito pupunha,St. Marche,53.3,,,0,,0,,


In [72]:
df.update(data_df)
df

,Product,Market,Product Price per Kg,URL,Product Name,Product Price,Product Unit,Product Size,Status,new URL
0,batata doce rosada,Sams Club,5.26,https://www.samsclub.com.br/batata-doce-roxa-p...,Batata Doce Roxa Pacote,5.26,kg,1,Same Price,https://www.samsclub.com.br/batata-doce-roxa-p...
1,batata doce rosada,Pao de Acucar,6.7,https://www.paodeacucar.com/produto/80528/bata...,Batata Doce Rosada 400g,2.68,g,400.0,Same Price,https://www.paodeacucar.com/produto/80528/bata...
2,batata doce rosada,Giga,3.9,,,,,,,
3,batata doce rosada,Oba,5.99,,,,,,,
4,batata doce rosada,Hakuo,5.59,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
120,quiabo,St. Marche,39.90,https://www.marche.com.br/produtos/quiabo-sele...,Quiabo Selecionado,39.90,kg,1.000,Same Price,
121,ervilha torta,Pao de Acucar,63.96,https://www.paodeacucar.com/produto/87149/ervi...,Ervilha Torta QUALITÁ Bandeja 250g,15.99,g,250.0,More Expensive,https://www.paodeacucar.com/produto/87149/ervi...
122,ervilha torta,St. Marche,34.63,https://www.marche.com.br/produtos/ervilha-con...,Ervilha Congelada BONDUELLE 300g,10.39,g,0.300,On Sale,https://www.marche.com.br/produtos/ervilha-con...
123,palmito pupunha,St. Marche,53.3,,,0,,0,,


In [73]:
df.to_csv(filename, index=False, sep=";", encoding = "utf-8")

In [97]:
df.sort_values(by=["Market"]).query("Status == 'On Sale'")

,Product,Market,Product Price per Kg,URL,Product Name,Product Price,Product Unit,Product Size,Status,new URL
84,pepino,Pao de Acucar,8.7,https://www.paodeacucar.com/produto/54512/pepi...,Pepino Japonês A Granel 300g,2.61,g,300.0,On Sale,https://www.paodeacucar.com/produto/54512/pepi...
93,couve,Pao de Acucar,8.97,https://www.paodeacucar.com/produto/3714/couve...,Couve-Manteiga Qualitá,2.99,un,3,On Sale,https://www.paodeacucar.com/produto/3714/couve...
107,couve flor,Pao de Acucar,18.97,https://www.paodeacucar.com/produto/170688/cou...,Couve Flor Florete Congelado D´AUCY Pacote 300g,5.69,g,300.0,On Sale,https://www.paodeacucar.com/produto/170688/cou...
124,abobora paulista,Pao de Acucar,5.69,https://www.paodeacucar.com/produto/80149/abob...,"Abóbora Paulista 1,1Kg",6.26,kg,1.1,On Sale,https://www.paodeacucar.com/produto/80149/abob...
66,abobora japonesa,Sams Club,4.27,https://www.samsclub.com.br/abobora-japonesa-3...,Abóbora Japonesa,4.27,kg,1,On Sale,https://www.samsclub.com.br/abobora-japonesa-3...
115,aspargos,Sams Club,,https://www.samsclub.com.br/aspargo-verde-impo...,Aspargo Verde Importado Talsa,32.98,,,On Sale,https://www.samsclub.com.br/aspargo-verde-impo...
69,agriao,Sams Club,,https://www.samsclub.com.br/agriao-hidroponico...,Agrião Hidropônicos Matsusako Maço,5.96,,,On Sale,https://www.samsclub.com.br/agriao-hidroponico...
74,rucula,Sams Club,0.00,https://www.samsclub.com.br/rucula-hidroponica...,Rúcula Hidropônica 1 Unidade,0,un,3,On Sale,https://www.samsclub.com.br/rucula-hidroponica...
106,couve flor,Sams Club,14.9,https://www.samsclub.com.br/couve-flor-florete...,Couve-flor Florete Congelada D'aucy Pacote 1kg,14.90,kg,1,On Sale,https://www.samsclub.com.br/couve-flor-florete...
83,pepino,Sams Club,7.98,https://www.samsclub.com.br/pepino-comum-pacot...,Pepino Comum Pacote,7.98,kg,1,On Sale,
